# Agent Chat with Multi-Modality Models

Here, we use LLaVA as an example.


## LLaVA Setup
Please follow the LLaVA Github [page](https://github.com/haotian-liu/LLaVA/) to install the LLaVA, download weights, and start the server.

For instance, here are some important steps:
```bash
# Download package
git clone https://github.com/haotian-liu/LLaVA.git
cd LLaVA

# Install inference package
conda create -n llava python=3.10 -y
conda activate llava
pip install --upgrade pip  # enable PEP 660 support
pip install -e .

# Download and serve the model
python -m llava.serve.model_worker --host 0.0.0.0 --controller http://localhost:10000 --port 40000 --worker http://localhost:40000 --model-path liuhaotian/llava-v1.5-7b
```

Some helpful packages and dependencies:
```bash
conda install -c nvidia cuda-toolkit
```


### Launch

In one terminal, start the controller
```bash
python -m llava.serve.controller --host 0.0.0.0 --port 10000
```


In another terminal, start the worker, which will load the model to GPU
```bash
python -m llava.serve.model_worker --host 0.0.0.0 --controller http://localhost:10000 --port 40000 --worker http://localhost:40000 --model-path liuhaotian/llava-v1.5-13b
```

**Note: make sure the environment of this notebook also installed the llava package from `pip install -e .`**

In [1]:
import requests
import json
from llava.conversation import default_conversation as conv
from llava.conversation import Conversation

[2023-10-13 14:36:14,868] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
# Setup some global constants for convenience
WORKER_ADDR = "http://0.0.0.0:40000"
CONTROLLER_ADDR = "http://0.0.0.0:10000"
SEP =  conv.sep
ret = requests.post(CONTROLLER_ADDR + "/list_models")
MODEL_NAME = ret.json()["models"][0]
print("Model Name:", MODEL_NAME)

Model Name: llava-v1.5-13b


In [20]:
from PIL import Image
from io import BytesIO
import base64


def get_image_data(image_file):
    if image_file.startswith('http://') or image_file.startswith('https://'):
        response = requests.get(image_file)
        content = response.content
    elif image_file.startswith("data:image/png;base64,"):
        return image_file.replace("data:image/png;base64,", "")
    else:
        image = Image.open(image_file).convert('RGB')
        content = open(image_file, "rb").read()
    return base64.b64encode(content).decode('utf-8')
    
def _to_pil(data):
    return Image.open(BytesIO(base64.b64decode(data)))


# _to_pil("...")


In [22]:

def llava_call(prompt:str, model_name: str=MODEL_NAME, images: list=[], seed=1) -> str:
    assert prompt.count("<image>") == len(images), "the number "
    "of image token in prompt and in the images list should be the same!"
    
    images = [get_image_data(x) for x in images]
    
    headers = {"User-Agent": "LLaVA Client"}
    pload = {
        "model": model_name,
        "prompt": prompt,
        "max_new_tokens": 1000,
        "temperature": 0.7,
        "stop": SEP,
        "images": images,
        "seed": seed
    }

    response = requests.post(WORKER_ADDR + "/worker_generate_stream", headers=headers,
            json=pload, stream=False)

    for chunk in response.iter_lines(chunk_size=8192, decode_unicode=False, delimiter=b"\0"):
        if chunk:
            data = json.loads(chunk.decode("utf-8"))
            output = data["text"].split(SEP)[-1]
    
    output = output.replace(prompt, "")
    
    return output.strip().rstrip()



Here is the image that we are going to use.

![Image](https://github.com/haotian-liu/LLaVA/raw/main/images/llava_logo.png)

In [23]:
out = llava_call("Describe this image: <image>", 
                 images=["https://github.com/haotian-liu/LLaVA/raw/main/images/llava_logo.png"])
print(out)

A toy shaped like a lizard, which has a flaming tail and red glasses.


In [8]:
out = llava_call("Describe the image in details so that I can reconstruct it in the future."
                 "Make sure you have extracted the important text, values, and numbers from the image: <image>", 
                 images=[
                 "https://th.bing.com/th/id/R.92724d713cb2d130cb03ddfea19303d8?rik=HBqBdBO28gmzIQ&pid=ImgRaw&r=0"
                 ])
print(out)

The image depicts a map of the United States in red, yellow, and green colors. The mainland states are displayed in shades of red and yellow, while the coastal states are colored green. The colors represent the relative distribution of demographic groups in each state, with red representing the Republican party, green representing the Democratic party, and yellow indicating a more even distribution. This map helps to visually represent and understand the political landscape of the United States, showcasing the demographics and affiliations of each state.


In [15]:
out = llava_call("What is in the image? What is the problem with it?: <image>", 
                 images=[
"https://www.lovethegarden.com/sites/default/files/content/articles/UK_moss-in-lawn-closeup.jpg"
                 ])
print(out)

In the image, the grass is green and lush. There is an insect sitting on the grass, which is also covered with green moss. The scene has a close-up view, emphasizing the intricate details of the insect and the surrounding greenery.


## AutoGen Integration: Garden Helper


Here we demonstrate a very simple multi-agent collaboration on creating visualization.

The user will upload an image of their garden, the image agent (with LLaVA backend) will read the image and describe the problem. Then, the suggestion agent (AssistantAgent with GPT model) will give suggestions on how to treat the problem.


Here, we found a problem in our garden and took a photo:
![](http://th.bing.com/th/id/R.105d684e5df7d540e61f6300d0bd374e?rik=PR8LCyvpe93DZA&pid=ImgRaw&r=0)

In [9]:

import autogen
from autogen import AssistantAgent

config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0314", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)


In [10]:
llm_config = {"config_list": config_list_gpt4, "seed": 42}


In [26]:
import pdb


class ImageAgent(AssistantAgent):

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def generate_reply(
        self,
        messages=None,
        sender=None,
        **kwargs,
    ):
        # Note: we did not use "llm_config" yet.
        # TODO: make the LLaVA design compatible with llm_config
        if all((messages is None, sender is None)):
            error_msg = f"Either {messages=} or {sender=} must be provided."
            logger.error(error_msg)
            raise AssertionError(error_msg)

        if messages is None:
            messages = self._oai_messages[sender]

        image_name = messages[-1]["content"]
        prompt = self.system_message + ": <image>"
        out = llava_call(prompt=prompt,
                         model_name=MODEL_NAME,
                         images=[image_name])

        print(out)
        return out


image_agent = ImageAgent(
    name="image-explainer",
    system_message="What is in the image? What is the problem with it?"
)

user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "groupchat"
    },
    human_input_mode="NEVER",
)
suggestion_giver = autogen.AssistantAgent(
    name=
    "Suggestion-Giver",
    system_message="Give me treatment suggestions for my garden! You can find the description of my image from the image-explainer agent. Keep the answer concise and short.",
    llm_config=llm_config,
)

groupchat = autogen.GroupChat(agents=[user_proxy, image_agent, suggestion_giver],
                              messages=[],
                              max_round=3)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)


In [27]:
user_proxy.initiate_chat(manager, 
                         message="http://th.bing.com/th/id/R.105d684e5df7d540e61f6300d0bd374e?rik=PR8LCyvpe93DZA&pid=ImgRaw&r=0")

User_proxy (to chat_manager):

http://th.bing.com/th/id/R.105d684e5df7d540e61f6300d0bd374e?rik=PR8LCyvpe93DZA&pid=ImgRaw&r=0

--------------------------------------------------------------------------------

image-explainer (to chat_manager):



--------------------------------------------------------------------------------
Suggestion-Giver (to chat_manager):

The image shows a small garden with an assortment of plants and flowers in different colors, including yellow, purple, and red. There is a gravel path, a small pond, lush greenery, and a stone lantern. Nearby, there are benches for people to sit and enjoy the view.

Based on the image, I suggest:
1. Pruning any overgrown plants to maintain shape and promote healthy growth.
2. Regularly inspect and monitor plants for pests or diseases.
3. Add compost or organic fertilizer to enhance soil nutrients.
4. Consider installing garden lights for better aesthetics during evening hours.
5. Ensure proper watering and avoid overwatering to 